In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sms spam detection/SMSSpamCollection", sep = '\t', names=['lable', 'message'])

In [ ]:
data.head()

,lable,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lable    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, random_state=0)

In [ ]:
test

,lable,message
0,ham,"Storming msg: Wen u lift d phne, u say ""HELLO""..."
1,spam,<Forwarded from 448712404000>Please CALL 08712...
2,ham,And also I've sorta blown him off a couple tim...
3,ham,"Sir Goodmorning, Once free call me."
4,ham,All will come alive.better correct any good lo...
...,...,...
1388,ham,No. On the way home. So if not for the long dr...
1389,spam,"Thanks for your ringtone order, ref number R83..."
1390,spam,Sorry! U can not unsubscribe yet. THE MOB offe...
1391,ham,Am I the only one who doesn't stalk profiles?


In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
corpus = []

In [ ]:
for i in range(0, len(train)):
  review = re.sub('[^a-zA-Z]', ' ', train['message'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
test_corpus = []
for i in range(0, len(test)):
  review = re.sub('[^a-zA-Z]', ' ', test['message'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  test_corpus.append(review)

In [ ]:
corpus

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_train = cv.fit_transform(corpus).toarray()

x_test = cv.transform(test_corpus).toarray()

In [ ]:
x_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# cv = TfidfVectorizer()
# x_train = cv.fit_transform(corpus).toarray()

# x_test = cv.transform(test_corpus).toarray()

In [ ]:
y_train = pd.get_dummies(train['lable'])
y_train = y_train.iloc[:, 1].values

y_test = pd.get_dummies(test['lable'])
y_test = y_test.iloc[:, 1].values

In [ ]:
y_test

array([0, 1, 0, ..., 1, 0, 0], dtype=uint8)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred, y_test)

In [ ]:
cm

array([[1204,   13],
       [   4,  172]])

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_pred, y_test)

In [ ]:
acc

0.9877961234745154

In [ ]:
import pickle

In [ ]:
pickle.dump(cv, open('sc.sav', 'wb'))

In [ ]:
pickle.dump(model, open('model.sav', 'wb'))

## Testing

In [ ]:
loaded_model = pickle.load(open('/content/sc.sav', 'rb'))

In [ ]:
model1 = pickle.load(open('/content/model.sav', 'rb'))

In [ ]:
m = "Free entry in 2 a wkly comp to win FA Cup fina..."

In [ ]:
test = []

In [ ]:
review = re.sub('[^a-zA-Z]', ' ', m)
review = review.lower()
review = review.split()
review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)
test.append(review)

In [ ]:
test

['free entri wkli comp win fa cup fina']

In [ ]:
new_test = loaded_model.transform(test).toarray()

In [ ]:
result = model1.predict(new_test)
print(result)

[1]
